In [ ]:
!git clone https://github.com/syncdoth/RetNet.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers==4.38.2

In [ ]:
!pip install timm

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from RetNet.retnet.modeling_retnet import RetNetModel
from RetNet.retnet.configuration_retnet import RetNetConfig

In [ ]:
num_users = 100
num_items = 100
interaction_matrix = np.random.rand(num_users, num_items)

In [ ]:
# Преобразование данных в формат torch.Tensor
interaction_tensor = torch.FloatTensor(interaction_matrix)

# Разделение данных на обучающий и тестовый наборы
train_data, test_data = train_test_split(interaction_tensor, test_size=0.2, random_state=42)

# Создание DataLoader для обучающего и тестового наборов
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

In [ ]:
class CollaborativeFilteringRetNet(nn.Module):
    def __init__(self, num_users, num_items, retnet_model, hidden_size):
        super(CollaborativeFilteringRetNet, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.retnet_model = retnet_model
        self.linear = nn.Linear(hidden_size, 1)  # Добавляем линейный слой

    def forward(self, user_idx, item_idx):
        # Объединяем user_idx и item_idx
        input_ids = torch.stack((user_idx, item_idx + self.num_users), dim=1)

        # Передача input_ids в модель RetNetModel
        outputs = self.retnet_model(input_ids=input_ids)

        # Получение последнего скрытого состояния из RetNetModel
        last_hidden_state = outputs.last_hidden_state[:, -1, :]  # Берем последний токен

        # Применение линейного слоя
        linear_output = self.linear(last_hidden_state)

        return linear_output.squeeze()  # Возвращаем выход линейного слоя

In [ ]:
# Инициализация RetNet
config = RetNetConfig(decoder_layers=8,
                      decoder_embed_dim=512,
                      decoder_value_embed_dim=1024,
                      decoder_retention_heads=4,
                      decoder_ffn_embed_dim=1024)
retnet_model = RetNetModel(config)

# Инициализация модели CollaborativeFilteringRetNet
model = CollaborativeFilteringRetNet(num_users, num_items, retnet_model, hidden_size=config.decoder_embed_dim)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()  # Используем Mean Squared Error в качестве функции потерь
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        optimizer.zero_grad()
        user_idx, item_idx = batch.nonzero().t()  # Получаем индексы ненулевых элементов
        predictions = model(user_idx, item_idx)
        target = batch[user_idx, item_idx].float()  # Преобразуем в float для совместимости с MSELoss
        loss = criterion(predictions, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/10, Loss: 4.055309732755025
Epoch 2/10, Loss: 0.44617387652397156
Epoch 3/10, Loss: 0.2547316253185272
Epoch 4/10, Loss: 0.1845973332722982
Epoch 5/10, Loss: 0.14536276956399283
Epoch 6/10, Loss: 0.11803701519966125
Epoch 7/10, Loss: 0.11357021580139796
Epoch 8/10, Loss: 0.10378706455230713
Epoch 9/10, Loss: 0.09797770033280055
Epoch 10/10, Loss: 0.09414791315793991


In [ ]:
# Оценка производительности модели на тестовом наборе данных
model.eval()
test_loss = 0.0
with torch.no_grad():
    for batch in test_loader:
        user_idx, item_idx = batch.nonzero().t()
        predictions = model(user_idx, item_idx)
        target = batch[user_idx, item_idx].float()
        loss = criterion(predictions, target)
        test_loss += loss.item()
print(f"Test Loss: {test_loss/len(test_loader)}")